In [ ]:
# default_exp utils

In [ ]:
#export
from datetime import datetime
from typing import Union, Any, Optional, List, Tuple, Iterator, Mapping, Set
path_type = Union[str, Any]
bin_val = Union[int, bytes]
bytes_encoded = Union[int, bytes]
datetime_object = datetime

In [ ]:
#export
def binary_file_generator(bin_file: path_type, marker: bytes = b'\x00UUUU', block_size: int = 4096) -> Iterator[bytes]:
    """
    str, bytes, int > bytes
    :param bin_file: arquivo binario que contém os dados
    :param marker: separador de blocos
    :param block_size: tamanho em bytes que é "lido" por vez no arquivo, evitando problemas de memória
    :return: bloco em formato binario

    Gerador que fornece a partir de de um arquivo binário, um bloco binário por vez.

    """
    with open(bin_file, mode='rb') as bfile:
        # O primeiro bloco do arquivo é o cabeçalho e tem 36 bytes de tamanho.
        yield bfile.read(36)
        # As demais partes podem prosseguir normalmente
        current = b''
        while True:
            block = bfile.read(block_size)
            if not block:  # end-of-file
                # yield current
                return None
            current += block
            while True:
                markerpos = current.find(marker)
                if markerpos < 0:
                    break
                yield current[:markerpos]
                current = current[markerpos + len(marker):]


def filtered_binary_file_generator(iterator: Iterator[bytes],
                                  **kwargs: Union[int, datetime_object, Tuple[float, float]]) -> Iterator[bytes]:
    """
    Gerador que lida com dados de GPS e filtra blocos fornecidos por outro gerador genérico.
    Filtros disponíveis: Tipo de bloco, threadID e coordenadas geográficas (retângulo).
    :param iterator: gerador que fornece um bloco em formato binario
    :param blocktype: número que indica o tipo de bloco
    :param threadid: número que indica o thread ID do bloco
    :param data_hora_inicio: dia, mês, ano, reserva
    :param data_hora_final: dia, mês, ano, reserva
    :param latitudes: latitude máxima, latitude mínima, em graus decmais
    :param longitudes: longitude mínima, longitude máxima, em graus decimais
    :param ignore_gps_fail: considera válido blocos sem marca de localização
    :return: bloco em formato binario
    """
    # str_form = "((%d, %m, %y, 0), (%H, %M, %S, 0))"
    blocktype: Union[int, Tuple[int]] = kwargs.get('blocktype', None)
    threadid: Union[int, Tuple[int]] = kwargs.get('threadid', None)
    data_hora_inicio: datetime_object = kwargs.get('data_hora_inicio', None)
    data_hora_final: datetime_object = kwargs.get('data_hora_final', None)
    latitudes: Tuple[float, float] = kwargs.get('latitudes', (90, -90))
    longitudes: Tuple[float, float] = kwargs.get('longitudes', (-180, 180))
    gps_updated: bool = kwargs.get('ignore_gps_fail', True)
    lat_long = 0, 0
    if type(blocktype) is int:
        blocktype = (blocktype, )
    if type(threadid) is int:
        threadid = (threadid, )
    for data in iterator:
        bloco = Block(data)
        if bloco.block_type == 40:  # dados de GPS
            lat_long = DataType40(data).latitude, DataType40(data).longitude  # atualiza a posição
            gps_updated = True
        # filtro de coordenas geográficas
        if not gps_updated:
            continue # ignora blocos até que seja fornecido a primeira posição
        if not latitudes[0] >= lat_long[0] >= latitudes[1] and longitudes[0] <= lat_long[1] <= longitudes[1]:
            continue  # ignora blocos fora de posição até nova atualização
        if not blocktype:
            blocktype = bloco.block_type
        if not threadid:
            threadid = bloco.thread_id
        if not data_hora_inicio:
            data_hora_inicio = datetime(1900, 1, 1, 00, 00, 00, 00)
        if not data_hora_final:
            data_hora_final = datetime.now()
        # filtro de datas e horas, tipo e thread ID
        if bloco.block_type in (63, 65):  # blocos com marca temporal
            carimbo = TimedBlock(data).datetime_stamp
            if Block(data).block_type in blocktype \
               and Block(data).thread_id in threadid \
               and data_hora_inicio < carimbo < data_hora_final:
                yield data
        # filtros de tipo e thread ID
        else:  # blocos sem marca temporal
            if Block(data).block_type == blocktype and Block(data).thread_id == threadid:
                yield data

def group_generator(iterator: Iterator[bytes], group: int) -> Iterator[List[bytes]]:
    """
    :param iter_gen: gerador que fornece um bloco único em formato binario
    :param group: quantos blocos devem ser agrupados, ou o tamanho da lista desejada na saída.
    :return: uma lista de blocos binários

    Um gerador que agrupa blocos binários fornecido por outro gerador em uma lista de tamanho conhecido.
    O tamanho do grupo (da lista agrupada) é definido por 'group'.
    Pode ser usado para facilitar redução no tamanho de matrizes de arquivos ou para agrupar amostras de blocos.
    """
    for n in iterator:
        mini = [n]
        for g in range(group - 1):
            mini.append(next(iterator, None))
        if None in mini:
            return
        # print(f'mini {len(mini)}, {len(mini[0])}')
        yield mini


def bin2dec(binary_data: bytes, is_signed: bool = True) -> int:
    """
    bytes, bool > int
    :param binary_data: valor de int comprimido
    :param is_signed: indica se é um valor negativo ou não
    :return: valor traduzido

    Conversor binario para int.
    """
    return int.from_bytes(binary_data, byteorder='little', signed=is_signed)


def bin2str(binary_data: bytes) -> str:
    """
    bytes > str
    :param binary_data: valor de str comprimida
    :return: str traduzida

    Conversor binario para str.
    Erros do 'decoder' são ignorados.
    Ignora o final do dado binario ('\x00') que é usado apenas para manter o tamanho dos campos.
    """
    return binary_data.decode(errors="ignore").rstrip('\x00')
    # return binary_data.decode().rstrip('\x00')


def bin2date(binary_data: bytes) -> Tuple[int, int, int, int]:
    """
    bytes > (int, int, int, int)
    :param binary_data: valor de data comprimido
    :return: dia, mês, ano, reserva

    Date is expressed as dd/mm/yy/null, i.e. 4 bytes
    """
    day = binary_data[:1]
    month = binary_data[1:2]
    year = binary_data[2:3]
    reserve = binary_data[3:]
    return bin2dec(day), bin2dec(month), bin2dec(year), bin2dec(reserve)


def bin2time(binary_data: bytes) -> Tuple[int, int, int, int]:
    """
    bytes > (int, int, int, int)
    :param binary_data: valor de hora comprimido
    :return: horas, minutos, segundos, décimos de segundo

     Time is expressed as hh/mm/ss/cc (4 bytes), where cc is centiseconds;
     and a 32-bit nanoseconds field, expressed as an unsigned 32-bit integer,
     to support higher precision where required.
     At most one of cc and nanoseconds can be nonzero.
    """
    hours = binary_data[:1]
    minutes = binary_data[1:2]
    seconds = binary_data[2:3]
    centiseconds = binary_data[3:]
    return bin2dec(hours), bin2dec(minutes), bin2dec(seconds), bin2dec(centiseconds)


def decode_spectrum(b: int, offset: int) -> float:
    """
    int, int > float
    :param b: valor comprimido
    :param offset: offset do valor comprimido
    :return: valor traduzido

    return  spectral power level in dBm,
        truncated if necessary to range [offset – 127.5, offset]

    b = stored byte values, in range [0, 255]
    offset = level offset in dBm,
        stored as a signed byte, range [-128, 127].
        A typical offset level is -20 dBm.
    """
    return b/2 + offset - 127.5


def decode_spectrum_bytes(b: bytes, offset: bytes) -> float:
    """
    byte, byte > float
    :param b: valor comprimido
    :param offset: offset do valor comprimido
    :return: valor traduzido

    return  spectral power level in dBm,
        truncated if necessary to range [offset – 127.5, offset]

    b = stored byte values, in range [0, 255]
    offset = level offset in dBm,
        stored as a signed byte, range [-128, 127].
        A typical offset level is -20 dBm.
    """
    b_int = bin2dec(b)
    offset_int = bin2dec(offset, False)
    return b_int/2 + offset_int - 127.5

In [1]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_main.ipynb.
Converted 01_parser.ipynb.
Converted 02_utils.ipynb.
Converted 03_test_run.ipynb.
Converted 04_driver.ipynb.
Converted index.ipynb.
